In [1]:
import sys
import networkx as nx
import os
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch_geometric.loader import NeighborLoader
import torch
import subprocess
from typing import Tuple
import socket
import time

# Necessary to import from sibling directory
sys.path.append("..")


from pymdb import (
    MDBClient,
    TrainGraphLoader,
    EvalGraphLoader,
    SamplingGraphLoader,
    Sampler,
)


In [2]:
## Path to MillenniumDB executables
SERVER_PYMDB_PATH = "/home/zeus/MDB/MillenniumDB-Dev/build/Release/bin/server_pymdb"
CREATE_DB_PATH = "/home/zeus/MDB/MillenniumDB-Dev/build/Release/bin/create_db"

## Port to run MillenniumDB server on
SERVER_PORT = 8080

In [14]:
## Generate graphs in NetworkX and MillenniumDB format. Returns the in-memory graph and the path to the on-disk graph.
def generate_graph(
    num_nodes: int,
    num_edges: int,
    num_node_feat: int,
) -> Tuple[Data, str]:
    # In-memory graph
    graph = from_networkx(
        nx.gnm_random_graph(num_nodes, num_edges, seed=2023, directed=True)
    )
    graph.node_feat = torch.rand(num_nodes, num_node_feat, dtype=torch.float32)
    graph.node_label = torch.randint(0, 2, (num_nodes,), dtype=torch.uint8)

    # On-disk MillenniumDB graph
    graph_fname = f"N{num_nodes}_E{num_edges}_F{num_node_feat}.milldb"
    with open(graph_fname, "w") as f:
        for idx in range(graph.num_nodes):
            f.write(
                f"N{idx} :L{graph.node_label[idx]} feat:{graph.node_feat[idx].tolist()}\n"
            )
        for edge in graph.edge_index.T:
            f.write(f"N{edge[0]}->N{edge[1]} :T\n")
    return graph, graph_fname


## Create a MillenniumDB database from a file in the current directory and return its path
def create_db(dump_path: str) -> str:
    dest_path = os.path.basename(dump_path).replace(".milldb", "")

    if os.path.isdir(dest_path):
        subprocess.run(["rm", "-rf", dest_path])

    result = subprocess.run(
        [CREATE_DB_PATH, dump_path, dest_path],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.PIPE,
    )
    if result.returncode != 0:
        raise RuntimeError(f"create_db: {result.stderr.decode('utf-8')}")
    return dest_path


## Start a MillenniumDB server for a given database directory
def start_server(db_path: str):
    process = subprocess.Popen(
        [SERVER_PYMDB_PATH, db_path, "-p", str(SERVER_PORT)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.PIPE,
    )

    # Wait for server to listen to port
    while socket.socket().connect_ex(("localhost", SERVER_PORT)) != 0:
        time.sleep(0.5)

    return process

## Kill a MillenniumDB server process and return its exit code
def kill_server(process) -> int:
    process.kill()
    return process.wait()

In [4]:
graph, mdb_graph_path = generate_graph(100, 200, 10)

In [31]:
create_db(mdb_graph_path)

'N100_E200_F10'

In [13]:

server_process = start_server("N100_E200_F10")
with MDBClient("localhost", SERVER_PORT) as client:
    pass
kill_server(server_process)

-9